# Дополнительные оценки качества моделей

## Импорт необходимых библиотек

In [1]:
import os
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from transformers import pipeline
from scipy.sparse import hstack
from datasets import Dataset
from copy import deepcopy
import regex
import re
import emoji

## Чтение тестовых данных

### Данные с чистыми сообщениями

`data/test_clear.json` – `.json` файл структуры `[{"text": "blablabla", "label": 0}]`, где ключ `text` представляет текстовое значение сообщения, а ключ `label` – класс сообщения (0: безопасное, 1: опасное).  
Сообщения были собраны из открытых Telegram чатов различных ВУЗов.

In [2]:
df_clear = pd.read_json('../data/test_clear.json')

In [3]:
df_clear.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25274 entries, 0 to 25273
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    25274 non-null  object
 1   label   25274 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 395.0+ KB


In [4]:
df_clear.head()

,text,label
0,"Здравствуйте, подскажите, пожалуйста, программ...",0
1,"Получается, все тоже самое, но теперь диплом т...",0
2,"Извините, а где можно узнать Актуальные маг пр...",0
3,"Добрый вечер, подскажите пожалуйста, есть ли з...",0
4,Здравствуйте! Только очная форма обучения,0


### Данные со спам сообщениями

`data/parsed_lols_2023.txt` – `.txt` файл с набором спам-сообщений.  
Файл был получен с [lols.bot](https://lols.bot/2023/)

In [5]:
parsed_lols_2023 = open('../data/parsed_lols_2023.txt', 'r', encoding='utf-8').readlines()

In [6]:
for i in range(len(parsed_lols_2023)):
    parsed_lols_2023[i] = parsed_lols_2023[i].replace('<br/>', '', 1).strip().replace('<br/>', '\n')

In [7]:
len(parsed_lols_2023)

32437

In [8]:
df_spam = pd.DataFrame()
df_spam['text'] = parsed_lols_2023
df_spam['label'] = [1] * len(parsed_lols_2023)

In [9]:
df_spam.head()

,text,label
0,🤝ƃАЗA ЧAT𝖮B ᎩCЛᎩГ🤝 + Б𝖮Т AВTОРАСCЫЛKИ🚀<br>\n❗️...,1
1,دانیالللل,1
2,🤙🏻🤘🏻Caмo𝚎 cлoжное нa сеpoм рынке — это найти н...,1
3,𝙇 𝘼 𝙈 𝙄 𝘼,1
4,🔲DROP SERVICE🔲<br><br>☕️ 𝔸ℝ𝔸𝔹𝕀ℂ𝔸×ℝ𝕆𝔹𝕌𝕊𝕋𝔸 ☕️ \n...,1


### Объединение данных

In [10]:
df_test = pd.concat([df_clear, df_spam])

In [11]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57711 entries, 0 to 32436
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    57711 non-null  object
 1   label   57711 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.3+ MB


In [12]:
df_test.head()

,text,label
0,"Здравствуйте, подскажите, пожалуйста, программ...",0
1,"Получается, все тоже самое, но теперь диплом т...",0
2,"Извините, а где можно узнать Актуальные маг пр...",0
3,"Добрый вечер, подскажите пожалуйста, есть ли з...",0
4,Здравствуйте! Только очная форма обучения,0


## Обработка данных

### Создание новых столбцов с числовыми данными

В спам сообщениях очень часто используются эмодзи, ссылки, теги (`@`), много новых строк (`\n`) и лишних пробелов, так что их количество в сообщении может играть большую роль в классификации сообщений

#### Функции для подготовки текста и дополнительных числовых данных

> Эти же функции находятся в `../utils/preprocessing.py`.

In [13]:
def count_emojis(text: str) -> int:
    """
    Считает количество эмодзи в тексте.

    Параметры:
        text (str): Исходный текст.

    Возвращает:
        int: Количество найденных эмодзи.
    """
    # Разбиваем текст на "графемные кластеры" (учитывая составные эмодзи)
    data = regex.findall(r'\X', text)
    # Считаем эмодзи
    emoji_counter = sum(emoji.is_emoji(word) for word in data)
    return emoji_counter

In [14]:
def count_newlines(text: str) -> int:
    """
    Считает количество символов новой строки (\n) в тексте.

    Параметры:
        text (str): Исходный текст.

    Возвращает:
        int: Количество символов новой строки.
    """
    # Подсчёт символов новой строки
    newline_count = text.count('\n')
    return newline_count

In [15]:
def count_whitespaces(text: str) -> int:
    """
    Считает количество последовательных пробелов (два или более) в тексте.

    Параметры:
        text (str): Исходный текст.

    Возвращает:
        int: Количество найденных последовательных пробелов.
    """
    # Считаем последовательности из двух и более пробелов
    extra_spaces_count = len(re.findall(r'\s{2,}', text))
    return extra_spaces_count

In [16]:
# Регулярное выражение для ссылок
url_pattern = r'(https?://[^\s]+|www\.[^\s]+|[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}(?:/[^\s]*)?)'

In [17]:
def count_links(text: str) -> int:
    """
    Считает количество ссылок в тексте.

    Параметры:
        text (str): Исходный текст.

    Возвращает:
        int: Количество найденных ссылок.
    """
    # Найти все ссылки в тексте
    links = re.findall(url_pattern, text)
    return len(links)

In [18]:
def count_tags(text: str) -> int:
    """
    Считает количество упоминаний (тегов) в тексте.

    Параметры:
        text (str): Исходный текст.

    Возвращает:
        int: Количество найденных тегов.
    """
    # Подсчёт символов "@" как маркеров тегов
    tag_count = text.count('@')
    return tag_count

Удаление пунктуации и эмодзи, приведение к нижнему регистру может снизить шум и сложность модели

In [19]:
def remove_emojis(text: str) -> str:
    """
    Удаляет все эмодзи из текста.

    Параметры:
        text (str): Исходный текст.

    Возвращает:
        str: Текст без эмодзи.
    """
    # Разбиваем текст на "графемные кластеры"
    data = regex.findall(r'\X', text)
    # Удаляем все эмодзи
    text_without_emojis = ''.join(word for word in data if not emoji.is_emoji(word))
    return text_without_emojis

Ссылки и тэги без пунктуации превращаются в набор латинских символов, что будет мешать работе модели, но при этом их нельзя удалять из текстов, так как это важные маркеры. Поэтому их необходимо заменить на что-то вроде `[LINK]` и `[TAG]`

In [20]:
def replace_links(text: str) -> str:
    """
    Заменяет все ссылки в тексте на "[LINK]".

    Параметры:
        text (str): Исходный текст.

    Возвращает:
        str: Текст с заменёнными ссылками.
    """
    # Заменить ссылки на [LINK]
    text = re.sub(url_pattern, '[LINK]', text)
    return text

In [21]:
def replace_tags(text: str) -> str:
    """
    Заменяет все упоминания (теги) в тексте на "[TAG]".

    Параметры:
        text (str): Исходный текст.

    Возвращает:
        str: Текст с заменёнными тегами.
    """
    # Регулярное выражение для тегов
    tag_pattern = r'@[a-zA-Z0-9_]+'
    # Замена тегов на [TAG]
    text = re.sub(tag_pattern, '[TAG]', text)
    return text

In [22]:
def preprocess_text(
        text: str,
        lower: bool = True,
        remove_punctuation: bool = True,
        remove_emoji: bool = True,
        remove_whitespaces: bool = True,
        remove_links: bool = True,
        remove_tags: bool = True
) -> str:
    """
    Предобрабатывает текст, включая нормализацию регистра, удаление пунктуации, эмодзи, ссылок, тегов и пробелов.

    Параметры:
        text (str): Исходный текст.
        lower (bool): Приводить ли текст к нижнему регистру (по умолчанию True).
        remove_punctuation (bool): Удалять ли пунктуацию (по умолчанию True).
        remove_emoji (bool): Удалять ли эмодзи (по умолчанию True).
        remove_whitespaces (bool): Удалять ли лишние пробелы (по умолчанию True).
        remove_links (bool): Удалять ли ссылки (по умолчанию True).
        remove_tags (bool): Удалять ли теги (по умолчанию True).

    Возвращает:
        str: Обработанный текст.
    """
    # Приведение текста к нижнему регистру
    if lower:
        text = text.lower()
    # Замена ссылок на [LINK]
    if remove_links:
        text = replace_links(text)
    # Замена тегов на [TAG]
    if remove_tags:
        text = replace_tags(text)
    # Удаление пунктуации
    if remove_punctuation:
        text = re.sub(r'[^\w\s]', '', text)  # Удаляет пунктуацию
        # Восстановление [LINK] после очистки пунктуации
        if remove_links:
            text = text.replace('LINK', '[LINK]')
        # Восстановление [TAG] после очистки пунктуации
        if remove_tags:
            text = text.replace('TAG', '[TAG]')
    # Удаление эмодзи
    if remove_emoji:
        text = remove_emojis(text)
    # Удаление лишних пробелов и символов новой строки
    if remove_whitespaces:
        text = re.sub(r'\s+', ' ', text).strip()
    return text

#### Подготовка текста и дополнительных числовых данных

In [23]:
emojis_count = []
newlines_count = []
whitespaces_count = []
links_count = []
tags_count = []
for text in df_test['text']:
    emojis_count.append(count_emojis(text))
    newlines_count.append(count_newlines(text))
    whitespaces_count.append(count_whitespaces(text))
    links_count.append(count_links(text))
    tags_count.append(count_tags(text))

In [24]:
df_test['emojis'] = emojis_count
df_test['newlines'] = newlines_count
df_test['whitespaces'] = whitespaces_count
df_test['links'] = links_count
df_test['tags'] = tags_count

In [25]:
df_test.head()

,text,label,emojis,newlines,whitespaces,links,tags
0,"Здравствуйте, подскажите, пожалуйста, программ...",0,0,3,2,1,0
1,"Получается, все тоже самое, но теперь диплом т...",0,0,0,0,0,0
2,"Извините, а где можно узнать Актуальные маг пр...",0,0,0,0,0,0
3,"Добрый вечер, подскажите пожалуйста, есть ли з...",0,0,0,0,0,0
4,Здравствуйте! Только очная форма обучения,0,0,0,0,0,0


In [26]:
texts = list(deepcopy(df_test['text']))
for text_i in range(len(texts)):
    temp = preprocess_text(texts[text_i])
    if not temp:
        temp = np.nan
    texts[text_i] = temp

In [27]:
texts

['здравствуйте подскажите пожалуйста программу магистерскую с университетом туши не убрали вот эту [LINK]',
 'получается все тоже самое но теперь диплом только российский',
 'извините а где можно узнать актуальные маг программы которые не приостановлены',
 'добрый вечер подскажите пожалуйста есть ли заочная форма обучения по направлению международная безопасность',
 'здравствуйте только очная форма обучения',
 'здравствуйте подскажите пожалуйста будут ли в этом году выдавать сертификаты на облегчённое поступление призёрампобедителям заключительного этапа всош по физкультуре',
 'если будут то какие права даёт этот сертификат спасибо',
 'здравствуйте скажите пожалуйста а когда будет следующий день открытых дверей и можно ли туда идти тем кто собирается в магистратуру',
 'from a friend',
 'здравствуйте надо ли на это мероприятие регистрироваться',
 'добрый день насколько увеличится количество бюджетных мест в этом году',
 'здравствуйте не нужно',
 'здравствуйте 200522 в 1900 будет ночь в 

In [28]:
len(texts)

57711

In [29]:
df_test['text_preprocessed'] = texts

In [30]:
df_test.head()

,text,label,emojis,newlines,whitespaces,links,tags,text_preprocessed
0,"Здравствуйте, подскажите, пожалуйста, программ...",0,0,3,2,1,0,здравствуйте подскажите пожалуйста программу м...
1,"Получается, все тоже самое, но теперь диплом т...",0,0,0,0,0,0,получается все тоже самое но теперь диплом тол...
2,"Извините, а где можно узнать Актуальные маг пр...",0,0,0,0,0,0,извините а где можно узнать актуальные маг про...
3,"Добрый вечер, подскажите пожалуйста, есть ли з...",0,0,0,0,0,0,добрый вечер подскажите пожалуйста есть ли зао...
4,Здравствуйте! Только очная форма обучения,0,0,0,0,0,0,здравствуйте только очная форма обучения


In [31]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57711 entries, 0 to 32436
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   text               57711 non-null  object
 1   label              57711 non-null  int64 
 2   emojis             57711 non-null  int64 
 3   newlines           57711 non-null  int64 
 4   whitespaces        57711 non-null  int64 
 5   links              57711 non-null  int64 
 6   tags               57711 non-null  int64 
 7   text_preprocessed  57252 non-null  object
dtypes: int64(6), object(2)
memory usage: 4.0+ MB


### Удаление лишних строк

In [32]:
sum(df_test.isnull().sum(axis = 1))

459

In [33]:
df_test.dropna(subset=['text_preprocessed'], inplace=True)

In [34]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57252 entries, 0 to 32436
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   text               57252 non-null  object
 1   label              57252 non-null  int64 
 2   emojis             57252 non-null  int64 
 3   newlines           57252 non-null  int64 
 4   whitespaces        57252 non-null  int64 
 5   links              57252 non-null  int64 
 6   tags               57252 non-null  int64 
 7   text_preprocessed  57252 non-null  object
dtypes: int64(6), object(2)
memory usage: 3.9+ MB


In [35]:
df_test.head()

,text,label,emojis,newlines,whitespaces,links,tags,text_preprocessed
0,"Здравствуйте, подскажите, пожалуйста, программ...",0,0,3,2,1,0,здравствуйте подскажите пожалуйста программу м...
1,"Получается, все тоже самое, но теперь диплом т...",0,0,0,0,0,0,получается все тоже самое но теперь диплом тол...
2,"Извините, а где можно узнать Актуальные маг пр...",0,0,0,0,0,0,извините а где можно узнать актуальные маг про...
3,"Добрый вечер, подскажите пожалуйста, есть ли з...",0,0,0,0,0,0,добрый вечер подскажите пожалуйста есть ли зао...
4,Здравствуйте! Только очная форма обучения,0,0,0,0,0,0,здравствуйте только очная форма обучения


In [36]:
count_zero = df_test[df_test['label'] == 0].shape[0]

In [37]:
count_zero

24830

In [38]:
count_one = df_test[df_test['label'] == 1].shape[0]

In [39]:
count_one

32422

Итого мы получили 1978 строк, из которых:
- 979 чистых сообщений
- 999 сообщений, помеченных как спам

## Загрузка моделей

In [40]:
models_path = "../models"  # Путь к папке с моделями

Загрузка scaler и vectorizer

In [41]:
with open(os.path.join(models_path, "scaler.pkl"), "rb") as scaler_file:
    scaler = pickle.load(scaler_file)

with open(os.path.join(models_path, "vectorizer.pkl"), "rb") as vectorizer_file:
    vectorizer = pickle.load(vectorizer_file)

Загрузка классических моделей

In [42]:
classic_models = {}
for file in os.listdir(models_path):
    if file.endswith(".pkl") and file not in ["scaler.pkl", "vectorizer.pkl"]:
        with open(os.path.join(models_path, file), "rb") as model_file:
            classic_models[file] = pickle.load(model_file)

Загрузка моделей rubert

In [43]:
rubert_models = {
    "rubert_tiny2": {
        "path": os.path.join(models_path, "finetuned_rubert_tiny2"),
    },
    "rubert_tiny2_p": {
        "path": os.path.join(models_path, "finetuned_rubert_tiny2_p"),
    },
}
for name, model_info in rubert_models.items():
    model_info["classifier"] = pipeline("text-classification", model=model_info["path"], truncation=True)

Device set to use cuda:0
Device set to use cuda:0


## Предсказание значений

### Функции для предсказания значений

Функция предсказания для классических моделей

In [44]:
def predict_message(text: str, vectorizer, scaler, model) -> tuple[int, list[float]]:
    """
    Предсказывает метку и вероятности для текста с использованием модели машинного обучения.

    Параметры:
        text (str): - входной текст для анализа
        vectorizer: - векторизатор для преобразования текста
        scaler: - масштабатор числовых признаков
        model: - модель машинного обучения

    Возвращает:
        tuple[int, list[float]] - метка предсказания и вероятности
    """
    # Предварительная обработка текста
    text_preprocessed = preprocess_text(text)

    # Векторизация текста
    text_vector = vectorizer.transform([text_preprocessed])

    # Создание числовых признаков
    numerical_features = pd.DataFrame(
        [[count_emojis(text), count_newlines(text), count_whitespaces(text), count_links(text), count_tags(text)]],
        columns=['emojis', 'newlines', 'whitespaces', 'links', 'tags']
    )
    # Масштабирование числовых признаков
    numerical_features_scaled = scaler.transform(numerical_features)

    # Объединение текстовых и числовых признаков
    features = hstack([text_vector, numerical_features_scaled])

    # Предсказание
    prediction = model.predict(features)
    probabilities = model.predict_proba(features)
    return int(prediction[0]), *probabilities.tolist()

Функция предсказания для моделей rubert

In [45]:
def bert_predict_batch(messages: list[str], classifier, threshold: float = 0.5) -> list[tuple[int, list[float]]]:
    """
    Функция для батчевого предсказания с помощью модели BERT через pipeline classifier.

    Параметры:
        messages (list[str]): – список сообщений для анализа
        classifier (pipeline): – pipeline классификатор
        threshold (float): – пороговое значение для предсказания класса

    Возвращает:
        list[tuple[int, list[float]]] - список предсказаний (0 или 1) и вероятностей
    """
    results = classifier(messages, batch_size=8)
    predictions = []
    for result in results:
        prediction = int(result['label'][-1])  # Извлечение метки предсказания
        score = result['score']  # Вероятность предсказания
        probabilities = [1 - score, score] if prediction == 1 else [score, 1 - score]

        if probabilities[1] < threshold:
            prediction = 0

        predictions.append((prediction, probabilities))

    return predictions

### Сбор предсказаний

In [46]:
# Подготовка данных для BERT моделей
dataset = Dataset.from_pandas(df_test)

for name, model_info in rubert_models.items():
    rubert_results = bert_predict_batch(list(dataset["text"]), model_info["classifier"], threshold=0.945)
    df_test[name] = [pred[0] for pred in rubert_results]

In [47]:
df_test.head()

,text,label,emojis,newlines,whitespaces,links,tags,text_preprocessed,rubert_tiny2,rubert_tiny2_p
0,"Здравствуйте, подскажите, пожалуйста, программ...",0,0,3,2,1,0,здравствуйте подскажите пожалуйста программу м...,0,0
1,"Получается, все тоже самое, но теперь диплом т...",0,0,0,0,0,0,получается все тоже самое но теперь диплом тол...,0,0
2,"Извините, а где можно узнать Актуальные маг пр...",0,0,0,0,0,0,извините а где можно узнать актуальные маг про...,0,0
3,"Добрый вечер, подскажите пожалуйста, есть ли з...",0,0,0,0,0,0,добрый вечер подскажите пожалуйста есть ли зао...,0,0
4,Здравствуйте! Только очная форма обучения,0,0,0,0,0,0,здравствуйте только очная форма обучения,0,0


### Сравнение

In [48]:
for name, model_info in rubert_models.items():
    print(name, '\n')
    print('Classification report:\n', classification_report(df_test['label'], df_test[name]))
    print("Confusion Matrix:\n", confusion_matrix(df_test['label'], df_test[name]))
    print('\n\n')

rubert_tiny2 

Classification report:
               precision    recall  f1-score   support

           0       0.97      0.99      0.98     24830
           1       1.00      0.97      0.98     32422

    accuracy                           0.98     57252
   macro avg       0.98      0.98      0.98     57252
weighted avg       0.98      0.98      0.98     57252

Confusion Matrix:
 [[24684   146]
 [  890 31532]]



rubert_tiny2_p 

Classification report:
               precision    recall  f1-score   support

           0       0.99      0.95      0.97     24830
           1       0.96      0.99      0.98     32422

    accuracy                           0.97     57252
   macro avg       0.98      0.97      0.97     57252
weighted avg       0.97      0.97      0.97     57252

Confusion Matrix:
 [[23551  1279]
 [  208 32214]]





**🔍 1. Обзор общей точности**

| Метрика        | rubert\_tiny2 | rubert\_tiny2\_p |
| -------------- | ------------- | ---------------- |
| Accuracy       | **0.98**      | 0.97             |
| Macro F1-score | **0.98**      | 0.97             |
| Weighted F1    | **0.98**      | 0.97             |

→ `rubert_tiny2` показывает **лучшую общую производительность**.

---

**📊 2. Классы по отдельности**

**Класс 0**:

| Метрика   | rubert\_tiny2 | rubert\_tiny2\_p |
| --------- | ------------- | ---------------- |
| Precision | 0.97          | **0.99**         |
| Recall    | **0.99**      | 0.95             |
| F1-score  | **0.98**      | 0.97             |

→ `rubert_tiny2` лучше **находит** все объекты класса 0 (Recall 0.99), а `rubert_tiny2_p` больше **точен**, но упускает часть объектов.

---

**Класс 1**:

| Метрика   | rubert\_tiny2 | rubert\_tiny2\_p |
| --------- | ------------- | ---------------- |
| Precision | **1.00**      | 0.96             |
| Recall    | 0.97          | **0.99**         |
| F1-score  | **0.98**      | 0.97             |

→ `rubert_tiny2` вообще не делает ложных положительных для класса 1 (Precision = 1.00), но немного теряет в Recall (пропускает 3%).

---

**🧮 Матрицы ошибок**

**rubert\_tiny2**:

```
[[24684   146]  → 133 false positive
 [  890 31532]] → 970 false negative
```

**rubert\_tiny2\_p**:

```
[[23551  1279]  → **1308 false positive** ← значительно больше
 [  208 32214]] → **402 false negative**
```

→ `rubert_tiny2_p` делает **намного больше ложных срабатываний** на класс 1 (принимает класс 0 за 1).
→ Но зато она **меньше пропускает** объектов класса 1 (Recall выше).

---

**✅ Вывод:**

| Критерий                      | Лучшая модель      |
| ----------------------------- | ------------------ |
| Общая точность                | `rubert_tiny2` ✅   |
| Поиск класса 0 (Recall)       | `rubert_tiny2` ✅   |
| Поиск класса 1 (Recall)       | `rubert_tiny2_p` ✅ |
| Точность класса 1 (Precision) | `rubert_tiny2` ✅   |
| Баланс (меньше FP + FN)       | `rubert_tiny2` ✅   |

🔹 **Если важнее избегать ложных срабатываний:** `rubert_tiny2`
🔹 **Если важнее не упустить ни одного объекта класса 1:** `rubert_tiny2_p` может быть предпочтительнее, но с оговорками

#### Строки с FN и FP для модели finetuned_rubert_tiny2

In [49]:
fn_rows = df_test[(df_test["label"] == 1) & (df_test["rubert_tiny2"] == 0)]
fp_rows = df_test[(df_test["label"] == 0) & (df_test["rubert_tiny2"] == 1)]

In [50]:
fn_rows.info()

<class 'pandas.core.frame.DataFrame'>
Index: 890 entries, 1 to 32436
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   text               890 non-null    object
 1   label              890 non-null    int64 
 2   emojis             890 non-null    int64 
 3   newlines           890 non-null    int64 
 4   whitespaces        890 non-null    int64 
 5   links              890 non-null    int64 
 6   tags               890 non-null    int64 
 7   text_preprocessed  890 non-null    object
 8   rubert_tiny2       890 non-null    int64 
 9   rubert_tiny2_p     890 non-null    int64 
dtypes: int64(8), object(2)
memory usage: 76.5+ KB


In [51]:
fn_rows.head(500)

,text,label,emojis,newlines,whitespaces,links,tags,text_preprocessed,rubert_tiny2,rubert_tiny2_p
1,دانیالللل,1,0,0,0,0,0,دانیالللل,0,1
33,معايا 2022,1,0,0,0,0,0,معايا 2022,0,1
60,هممونو,1,0,0,0,0,0,هممونو,0,1
72,B1 به بالا,1,0,0,0,0,0,b1 به بالا,0,1
78,Saba بیا,1,0,0,0,0,0,saba بیا,0,1
...,...,...,...,...,...,...,...,...,...,...
12042,𝒉𝒂,1,0,0,0,0,0,𝒉𝒂,0,1
12045,𝒉𝒆𝒍𝒍𝒐,1,0,0,0,0,0,𝒉𝒆𝒍𝒍𝒐,0,1
12048,𝒕𝒉𝒆𝒌,1,0,0,0,0,0,𝒕𝒉𝒆𝒌,0,1
12049,𝑯𝒊,1,0,0,0,0,0,𝑯𝒊,0,1


In [52]:
fp_rows.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146 entries, 596 to 25269
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   text               146 non-null    object
 1   label              146 non-null    int64 
 2   emojis             146 non-null    int64 
 3   newlines           146 non-null    int64 
 4   whitespaces        146 non-null    int64 
 5   links              146 non-null    int64 
 6   tags               146 non-null    int64 
 7   text_preprocessed  146 non-null    object
 8   rubert_tiny2       146 non-null    int64 
 9   rubert_tiny2_p     146 non-null    int64 
dtypes: int64(8), object(2)
memory usage: 12.5+ KB


In [53]:
fp_rows.head()

,text,label,emojis,newlines,whitespaces,links,tags,text_preprocessed,rubert_tiny2,rubert_tiny2_p
596,"Assalomu aleykum aka ,umumiy qancha summa keta...",0,0,0,1,0,0,assalomu aleykum aka umumiy qancha summa ketar...,1,1
653,milana56orsk@gmail.com,0,0,0,0,1,1,milana56orsk[LINK],1,1
802,Швырков Владислав Сергеевич shvyrkov.vs@gmail.com,0,0,0,0,2,1,швырков владислав сергеевич [LINK][LINK],1,0
1492,"Гасымова Нурана Назимовна 89393662727, nunu.ga...",0,0,0,0,2,1,гасымова нурана назимовна 89393662727 [LINK]19...,1,0
1929,"Хочется такую же, взять!!@",0,0,0,1,0,1,хочется такую же взять,1,1


In [54]:
fp_rows[['text', 'label']].to_dict('records')

[{'text': 'Assalomu aleykum aka ,umumiy qancha summa ketarkan  tooshirishga rudnga',
  'label': 0},
 {'text': 'milana56orsk@gmail.com', 'label': 0},
 {'text': 'Швырков Владислав Сергеевич shvyrkov.vs@gmail.com', 'label': 0},
 {'text': 'Гасымова Нурана Назимовна 89393662727, nunu.gasimova1999@gmail.com',
  'label': 0},
 {'text': 'Хочется такую же,  взять!!@', 'label': 0},
 {'text': 'РУДН жди меня 😂', 'label': 0},
 {'text': 'Написать на почту: mfc@rudn.ru', 'label': 0},
 {'text': 'mfc@rudn.ru', 'label': 0},
 {'text': 'Здравствуйте,я взял кредит и мне его одобрили, когда у меня спишется оплата за 1 год?',
  'label': 0},
 {'text': 'Кто может помочь подать это все за деньги ?', 'label': 0},
 {'text': 'Находиться на сайте РУДН - одно сплошное удовольствие', 'label': 0},
 {'text': 'https://vk.com/abi_pfur_mi_2022', 'label': 0},
 {'text': 'Беседа мехмата в ТГ: https://t.me/+wrA-Fcx_PYA0YTYy', 'label': 0},
 {'text': 'Где общаться со "своими"?\n💬 https://vk.cc/cfwcpn', 'label': 0},
 {'text': 'He